# DEM_for_J2_plasticity
## 1. 模型简介
这是一个基于深度能量法 (DEM，DPINN能量形式)的经典弹塑性求解框架。

**深度能量方法（Deep Energy Method, DEM）** 是一种基于变分原理的物理信息神经网络（PINN），通过最小化系统总自由能直接求解弹塑性边值问题。核心特点包括：
- **能量驱动**：以系统总自由能（弹性应变能+硬化能）作为损失函数，仅需位移场的一阶空间梯度。
- **路径依赖性处理**：结合径向返回算法（Radial Return Algorithm）更新塑性内部状态变量（塑性应变、等效塑性应变、背应力等），满足Kuhn-Tucker一致性条件。
- **网格兼容性**：支持结构化/非结构化网格，利用有限元形函数梯度计算位移梯度，提升大变形下的鲁棒性。
- **高效推断**：支持在粗网格训练后，直接迁移到细网格进行快速推断（GPU加速）。

## 2. 软件环境准备

### 2.1 基于dtk适配软件
<div class="alert alert-warning"> WARNING：镜像中环境已配置，此步骤省略 </div>

### 2.2 软件环境检查


In [1]:
# 检查torch版本
import torch
import os
import onescience

version = torch.__version__
num = float(version[:3])
# assert num == 1.10

# 检查硬件环境
device = "cpu"
if os.system('rocm-smi 2>/dev/null || hy-smi 2>/dev/null')==0:
    device = "dtk"

elif os.system('nvidia-smi 2>/dev/null')==0: 
    device = "cuda"

print("torch version:", version)
print("device =", device)
print("onescience version:", onescience.__version__)


============================ System Management Interface =============================
HCU     Temp     AvgPwr     Perf     PwrCap     VRAM%      HCU%      Mode     
0       48.0C    30.0W      auto     300.0W     0%         0%        N/A      
1       49.0C    23.0W      auto     300.0W     0%         0%        N/A      
2       50.0C    24.0W      auto     300.0W     0%         0%        N/A      
3       50.0C    23.0W      auto     300.0W     0%         0%        N/A      
=================================== End of SMI Log ===================================

torch version: 2.4.1
device = dtk
onescience version: 0.1.0


### 2.2 软件依赖安装


<div class="alert alert-warning"> WARNING: 镜像中环境已配置，此步骤省略 </div>

<div class="alert alert-note" style="color: blue;">
Note: 存在部分文件更新，请卸载当前环境下的 onescience 包，并安装当前目录的 whl 包。
    
请在最顶层目录(包含setup.py文件的目录)，通过命令行，执行下述安装指令：
</div>

In [ ]:
!pip install -e .

## 3. 素材准备
### 3.1 数据集准备
三个案例所需的数据均已放在对应的`Example`文件夹下：

```
Example1/
├── Shear.inp                    # Abaqus输入文件，定义矩形板周期性剪切问题的有限元模型
├── shear10_iso_Abaqus.npy       # 各向同性硬化（Isotropic）模型的Abaqus仿真结果，表现弹塑性滞回行为
├── shear10_kine_Abaqus.npy      # 运动硬化（Kinematic）模型的Abaqus仿真结果，考虑背应力演化
├── shearWave_iso_Abaqus.npy     # 各向同性硬化剪切波响应的Abaqus仿真结果
└── ShearWave_kine_Abaqus.npy    # 运动硬化剪切波响应的Abaqus仿真结果

Example2/
├── BiMat_Abaqus.npy             # 双材料板剪切行为的Abaqus仿真结果
└── BiMat.inp                   # Abaqus输入文件，构建异质双材料板有限元模型

Example3/
├── Hole_Abaqus.npy              # 3D标准带孔板的Abaqus仿真结果
└── Hole.inp                    # 带孔板标准Abaqus输入文件
```

## 4 训练


使用如下命令可以开启训练`--example`参数可以选择训练的案例，默认为Example1

In [ ]:
!python Plasticity_DEM_Train.py --example=1 

## 5 推理

使用如下命令可以开启训练`--example`参数可以选择训练的案例，默认为Example1

In [ ]:
python Plasticity_DEM_Inference.py --example=1

训练和推理完成后都会在result文件夹下生成vtk文件，可以放入Paraview中进行可视化处理